<a href="https://colab.research.google.com/github/blackgreenc/java/blob/master/Tensorflow_Hub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tensorflow-hub

In [ ]:
# dogs and cats image data import
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

raw_train, raw_test = tfds.load(
    'cats_vs_dogs',
    split = ['train[:80%]', 'train[20%:]'],
    as_supervised = True
)

Shuffling and writing examples to /root/tensorflow_datasets/cats_vs_dogs/4.0.0.incompleteCW2YYE/cats_vs_dogs-train.tfrecord
Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.0. Subsequent calls will reuse this data.


In [ ]:
# dogs and cats image data preprocessing
import numpy as np
import tensorflow as tf

def preprocess(image, label):
    out_image = tf.image.resize(image/255, [224,224])
    return out_image, label

batch_size = 32
train_batch = raw_train.map(preprocess).batch(batch_size)
test_batch = raw_test.map(preprocess).batch(batch_size)

In [ ]:
# model define
import tensorflow_hub as hub
url = "https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/5"

hub_model_transfer = tf.keras.Sequential([
    hub.KerasLayer(url, input_shape=(224,224,3), trainable = False), # 추가로 학습안할 거니까 False
    tf.keras.layers.Dense(1) # 이진분류니까 Dense 레이어 추가
])

In [ ]:
# tensorflow hub model compiling and training
hub_model_transfer.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy'])

hub_model_transfer.fit(train_batch, epochs = 5)

Epoch 1/5
582/582 [==============================] - 492s 839ms/step - loss: 0.3459 - accuracy: 0.9623
Epoch 2/5
582/582 [==============================] - 485s 832ms/step - loss: 0.2139 - accuracy: 0.9801
Epoch 3/5
582/582 [==============================] - 482s 828ms/step - loss: 0.2862 - accuracy: 0.9742
Epoch 4/5
582/582 [==============================] - 503s 865ms/step - loss: 0.2224 - accuracy: 0.9805
Epoch 5/5
582/582 [==============================] - 500s 859ms/step - loss: 0.2328 - accuracy: 0.9798


In [ ]:
hub_model_transfer.evaluate(test_batch, verbose=2)